***GENERATED CODE FOR dataprepbin PIPELINE.***

***DON'T EDIT THIS CODE.***

***CONNECTOR FUNCTIONS TO READ DATA.***

In [ ]:
import os
import datetime
import logging
import warnings
warnings.filterwarnings('ignore')
logging.basicConfig(format='%(levelname)s:%(message)s', level=logging.INFO)


class HDFSConnector:

    def fetch(spark, config):
        ################### INPUT HADOOP HOST PORT TO CONNECT WITH ###############################
        hdfs_server = str(os.environ['HDFS_SERVER'])
        hdfs_port = int(os.environ['HDFS_PORT'])
        df = spark.read.options(header='true', inferschema='true').csv(
            f"hdfs://{hdfs_server}:{hdfs_port}{eval(config)['url']}", header='true')
        display(df.limit(2).toPandas())
        return df

    def put(df, spark, config):
        return df.write.format('csv').options(header='true' if eval(config)["is_header"] == "Use Header Line" else 'false',
                                              delimiter=eval(config)["delimiter"]).save(("%s %s") % (datetime.datetime.now().strftime("%Y-%m-%d %H.%M.%S")+"_", eval(config)['url']))


***TRANSFORMATIONS FUNCTIONS THAT WILL BE APPLIED ON DATA***

In [ ]:
import json
from pyspark.ml.feature import Bucketizer
from pyspark.sql.functions import col, when
from pyspark.sql.functions import mean, stddev, min, max, col


class CleanseData:
    # def __init__(self,df):
    #     #print()

    def cleanValueForFE(self, value):
        if value == None:
            return ""
        elif str(value) == 'nan':
            return "nan"
        else:
            return value

    def replaceByMean(self, feature, df, mean_=-1):
        df1 = df
        df1 = df1.dropna()
        meanValue = self.cleanValueForFE(df1.select(
            mean(col(feature.name)).alias('mean')).collect()[0]["mean"])
        df = df.fillna(meanValue, subset=[feature.name])
        df.withColumn(feature.name, when(col(feature.name) == " ",
                      meanValue).otherwise(col(feature.name).cast("Integer")))
        return df

    def replaceByMax(self, feature, df, max_=-1):
        df1 = df
        df1 = df1.dropna()
        maxValue = self.cleanValueForFE(df1.select(
            max(col(feature.name)).alias('max')).collect()[0]["max"])
        df = df.fillna(maxValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", maxValue).otherwise(col(feature.name)))
        return df

    def replaceByMin(self, feature, df, min_=-1):
        df1 = df
        df1 = df1.dropna()
        minValue = self.cleanValueForFE(df1.select(
            min(col(feature.name)).alias('min')).collect()[0]["min"])
        df = df.fillna(minValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", minValue).otherwise(col(feature.name)))
        return df

    def replaceByStandardDeviation(self, feature, df, stddev_=-1):
        df1 = df
        df1 = df1.dropna()
        stddevValue = self.cleanValueForFE(df1.select(
            stddev(col(feature.name)).alias('stddev')).collect()[0]["stddev"])
        df = df.fillna(stddevValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", stddevValue).otherwise(col(feature.name)))
        return df

    def replaceDateRandomly(self, feature, df):
        df1 = df
        df1 = df1.dropna()
        fillValue = self.cleanValueForFE(
            df.where(col(feature.name).isNotNull()).head(1)[0][feature.name])
        df = df.fillna(str(fillValue), subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", fillValue).otherwise(col(feature.name)))
        # print("CleanseData:replaceDateRandomly Schema : ", df.#printSchema())
        return df

    def replaceNullValues(self, fList, df):
        featuresList = df.schema.fields
        for featureObj in fList:
            for feat in featuresList:
                if featureObj["feature"] in feat.name:
                    featureName = feat
                    if "mean" in featureObj["replaceby"]:
                        df = self.replaceByMean(featureName, df)
                    elif "max" in featureObj["replaceby"]:
                        df = self.replaceByMax(featureName, df)
                    elif "min" in featureObj["replaceby"]:
                        df = self.replaceByMin(featureName, df)
                    elif "stddev" in featureObj["replaceby"]:
                        df = self.replaceByStandardDeviation(featureName, df)
                    elif "random" in featureObj["replaceby"]:
                        df = self.replaceDateRandomly(featureName, df)
        return df


def bucketizerTransform(df, params, transformationData={}):
    dfReturn = df
    transform_params = params
    feature = transform_params['feature']

    dfReturn = dfReturn.fillna({feature: '0.0'})
    splits = []
    for bucket in transformationData["buckets"]:
        if float(bucket["lower_value"]) not in splits:
            splits.append(float(bucket["lower_value"]))
        if float(bucket["upper_value"]) not in splits:
            splits.append(float(bucket["upper_value"]))

    outcol = feature + "_bucketizer"
    bucketizer = Bucketizer(splits=splits, inputCol=feature, outputCol=outcol)
    bucketized = bucketizer.transform(dfReturn)
    dfReturn = bucketized
    dfReturn = dfReturn.withColumn(feature, round(dfReturn[feature], 2))

    return dfReturn


class TransformationMain:
    # TODO: change df argument in run with following
    def run(transformationDF, config):
        configObj = json.loads(config)
        featureData = configObj["FE"]
        transformationDF = CleanseData().replaceNullValues(featureData, transformationDF)
        transformationDF = bucketizerTransform(transformationDF, {'transformationsData': [{'feature_label': 'SURV_TM', 'buckets': [{'lower_value': '-Infinity', 'upper_value': '5'}, {'lower_value': '5', 'upper_value': '6'}, {'lower_value': '6', 'upper_value': '7'}, {'lower_value': '7', 'upper_value': '8'}, {'lower_value': '8', 'upper_value': '9'}, {'lower_value': '9', 'upper_value': '+Infinity'}], 'transformation_label': 'Bucketizer'}], 'feature': 'SURV_TM', 'type': 'real', 'selected': 'True', 'replaceby': 'mean', 'stats': {
                                               'count': '222', 'mean': '6.76', 'stddev': '2.0', 'min': '0.0', 'max': '10.0', 'missing': '0'}, 'transformation': [{'transformation': 'Bucketizer', 'selectedAsDefault': 1}], 'updatedLabel': 'SURV_TM'}, {'feature_label': 'SURV_TM', 'buckets': [{'lower_value': '-Infinity', 'upper_value': '5'}, {'lower_value': '5', 'upper_value': '6'}, {'lower_value': '6', 'upper_value': '7'}, {'lower_value': '7', 'upper_value': '8'}, {'lower_value': '8', 'upper_value': '9'}, {'lower_value': '9', 'upper_value': '+Infinity'}], 'transformation_label': 'Bucketizer'})
        transformationDF = transformationDF.drop('SURV_TM')
        display(transformationDF.limit(2).toPandas())
        return transformationDF


***CONNECTOR FUNCTIONS TO WRITE DATA.***

In [ ]:
import datetime
import requests
import datetime
import logging
import warnings
warnings.filterwarnings('ignore')
logging.basicConfig(format='%(levelname)s:%(message)s', level=logging.INFO)


class NumtraConnector:

    def put(inStages, inStagesData, stageId, spark, config):
        path = eval(config)['server_url']
        baseType = eval(config)['baseType']
        results_url = eval(config)['results_url']
        server = eval(config)['server']
        originalfile = eval(config)['orignalKey']
        eval(config)['pathOnly']
        filename = eval(config)['filename']
        eval(config)['ser']
        eval(config)['user']
        eval(config)['password']
        eval(config)['authSource']
        eval(config)['user_id']
        eval(config)['parent_id']
        eval(config)['project_id']
        time = str(int(datetime.datetime.now().timestamp()))

        inStagesData[inStages[0]]

        print(path)
        print(baseType)
        print(results_url)
        print(server)
        print(originalfile)
        print(filename)

        args = {
            'url': path,
            'baseType': baseType,
            'originalfile': originalfile,
            'filename': time + filename
        }

        response = requests.post(results_url, args)
        return response


***READING DATAFRAME***

In [ ]:
############## CREATE SPARK SESSION ############################ ENTER YOUR SPARK MASTER IP AND PORT TO CONNECT TO SERVER ################
from pyspark.sql import SparkSession
spark = SparkSession.builder.master('local[1]').getOrCreate()
#%run dataprepbinHooks.ipynb
try:
	#sourcePreExecutionHook()

	tumormarkerdataet = HDFSConnector.fetch(spark, "{'url': '/FileStore/platform/testdata/1709558123560_TumorMarkerdataset.csv', 'filename': 'TumorMarkerdataset.csv', 'delimiter': ',', 'file_type': 'Delimeted', 'dbfs_token': '', 'dbfs_domain': '', 'FilePath': '/Datasets/TumorMarkerdataset.csv', 'viewFileName': 'TumorMarkerdataset.csv', 'is_header': 'Use Header Line', 'baseType': 'hdfs', 'server_url': '/nexusMax/NexusMaxPlatform/uploads/platform/', 'results_url': 'http://dnm.bfirst.ai:44040/api/read/hdfs'}")

except Exception as ex: 
	logging.error(ex)
#spark.stop()


***TRANSFORMING DATAFRAME***

In [ ]:
#%run dataprepbinHooks.ipynb
try:
	#transformationPreExecutionHook()

	autofe = TransformationMain.run(tumormarkerdataet,json.dumps( {"FE": [{"transformationsData": [{"transformation_label": "novalue"}], "feature": "MAR_STAT", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "222", "mean": "0.35", "stddev": "0.48", "min": "0", "max": "1", "missing": "0"}, "updatedLabel": "MAR_STAT"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "RACE", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "222", "mean": "0.9", "stddev": "0.3", "min": "0", "max": "1", "missing": "0"}, "updatedLabel": "RACE"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "AGE_DX", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "222", "mean": "37.11", "stddev": "9.51", "min": "17", "max": "71", "missing": "0"}, "updatedLabel": "AGE_DX"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "DATE_yr", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "222", "mean": "2000.69", "stddev": "1.58", "min": "1998", "max": "2003", "missing": "0"}, "updatedLabel": "DATE_yr"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "HISTO3V", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "222", "mean": "0.98", "stddev": "0.15", "min": "0", "max": "1", "missing": "0"}, "updatedLabel": "HISTO3V"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "EOD10_SZ", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "222", "mean": "49.08", "stddev": "27.32", "min": "1", "max": "160", "missing": "0"}, "updatedLabel": "EOD10_SZ"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "EOD10_EX", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "222", "mean": "18.79", "stddev": "19.25", "min": "10", "max": "99", "missing": "0"}, "updatedLabel": "EOD10_EX"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "TUMOR_1V", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "222", "mean": "1.23", "stddev": "1.03", "min": "0", "max": "4", "missing": "0"}, "updatedLabel": "TUMOR_1V"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "TUMOR_2V", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "222", "mean": "1.97", "stddev": "1.65", "min": "0", "max": "6", "missing": "0"}, "updatedLabel": "TUMOR_2V"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "TUMOR_3V", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "222", "mean": "2.09", "stddev": "2.05", "min": "0", "max": "6", "missing": "0"}, "updatedLabel": "TUMOR_3V"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "SURGPRIM", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "222", "mean": "42.39", "stddev": "10.3", "min": "30", "max": "80", "missing": "0"}, "updatedLabel": "SURGPRIM"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "RADIATN", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "222", "mean": "1.47", "stddev": "1.55", "min": "1", "max": "7", "missing": "0"}, "updatedLabel": "RADIATN"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "RAD_SURG", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "222", "mean": "2.82", "stddev": "0.74", "min": "0", "max": "4", "missing": "0"}, "updatedLabel": "RAD_SURG"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "o_dth_cl", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "222", "mean": "0.09", "stddev": "0.29", "min": "0", "max": "1", "missing": "0"}, "updatedLabel": "o_dth_cl"}, {"transformationsData": [{"feature_label": "SURV_TM", "buckets": [{"lower_value": "-Infinity", "upper_value": "5"}, {"lower_value": "5", "upper_value": "6"}, {"lower_value": "6", "upper_value": "7"}, {"lower_value": "7", "upper_value": "8"}, {"lower_value": "8", "upper_value": "9"}, {"lower_value": "9", "upper_value": "+Infinity"}], "transformation_label": "Bucketizer"}], "feature": "SURV_TM", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "222", "mean": "6.76", "stddev": "2.0", "min": "0.0", "max": "10.0", "missing": "0"}, "transformation": [{"transformation": "Bucketizer", "selectedAsDefault": 1}], "updatedLabel": "SURV_TM"}]}))

	#transformationPostExecutionHook(autofe)

except Exception as ex: 
	logging.error(ex)


***WRITING DATAFRAME***

In [ ]:
#%run dataprepbinHooks.ipynb
try:
	#sinkPreExecutionHook()

	datasetsoutput = NumtraConnector.fetch(spark, "{'samplefile': '/FileStore/platform/sampleData/65f0440861584f9e85fc3866/part-00000-de0c52b9-b066-4325-a303-b901b84bae11-c000.csv', 'samplecount': 248, 'originalcount': 2226, 'orignalKey': '/FileStore/platform/testdata/1709558123560_TumorMarkerdataset.csv', 'pathOnly': '/Datasets', 'project_id': '65e92baca6cc707fe19dc67a', 'parent_id': '65e92baca6cc707fe19dc67a', 'original_schema': [{'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '65f044b261584f9e85fc3921', 'field': 'MAR_STAT', 'alias': 'MAR_STAT', 'type': 'numeric', 'position': '0'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '65f044b261584f9e85fc3922', 'field': 'RACE', 'alias': 'RACE', 'type': 'numeric', 'position': '1'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '65f044b261584f9e85fc3923', 'field': 'AGE_DX', 'alias': 'AGE_DX', 'type': 'numeric', 'position': '2'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '65f044b261584f9e85fc3924', 'field': 'DATE_yr', 'alias': 'DATE_yr', 'type': 'numeric', 'position': '3'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '65f044b261584f9e85fc3925', 'field': 'HISTO3V', 'alias': 'HISTO3V', 'type': 'numeric', 'position': '4'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '65f044b261584f9e85fc3926', 'field': 'EOD10_SZ', 'alias': 'EOD10_SZ', 'type': 'numeric', 'position': '5'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '65f044b261584f9e85fc3927', 'field': 'EOD10_EX', 'alias': 'EOD10_EX', 'type': 'numeric', 'position': '6'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '65f044b261584f9e85fc3928', 'field': 'TUMOR_1V', 'alias': 'TUMOR_1V', 'type': 'numeric', 'position': '7'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '65f044b261584f9e85fc3929', 'field': 'TUMOR_2V', 'alias': 'TUMOR_2V', 'type': 'numeric', 'position': '8'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '65f044b261584f9e85fc392a', 'field': 'TUMOR_3V', 'alias': 'TUMOR_3V', 'type': 'numeric', 'position': '9'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '65f044b261584f9e85fc392b', 'field': 'SURGPRIM', 'alias': 'SURGPRIM', 'type': 'numeric', 'position': '10'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '65f044b261584f9e85fc392c', 'field': 'RADIATN', 'alias': 'RADIATN', 'type': 'numeric', 'position': '11'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '65f044b261584f9e85fc392d', 'field': 'RAD_SURG', 'alias': 'RAD_SURG', 'type': 'numeric', 'position': '12'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '65f044b261584f9e85fc392e', 'field': 'o_dth_cl', 'alias': 'o_dth_cl', 'type': 'numeric', 'position': '13'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '65f044b261584f9e85fc392f', 'field': 'SURV_TM_bucketizer', 'alias': 'SURV_TM_bucketizer', 'type': 'real', 'position': '14'}], 'actual_schema': [{'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '65f044b261584f9e85fc3912', 'field': 'MAR_STAT', 'alias': 'MAR_STAT', 'type': 'numeric', 'position': '0'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '65f044b261584f9e85fc3913', 'field': 'RACE', 'alias': 'RACE', 'type': 'numeric', 'position': '1'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '65f044b261584f9e85fc3914', 'field': 'AGE_DX', 'alias': 'AGE_DX', 'type': 'numeric', 'position': '2'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '65f044b261584f9e85fc3915', 'field': 'DATE_yr', 'alias': 'DATE_yr', 'type': 'numeric', 'position': '3'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '65f044b261584f9e85fc3916', 'field': 'HISTO3V', 'alias': 'HISTO3V', 'type': 'numeric', 'position': '4'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '65f044b261584f9e85fc3917', 'field': 'EOD10_SZ', 'alias': 'EOD10_SZ', 'type': 'numeric', 'position': '5'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '65f044b261584f9e85fc3918', 'field': 'EOD10_EX', 'alias': 'EOD10_EX', 'type': 'numeric', 'position': '6'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '65f044b261584f9e85fc3919', 'field': 'TUMOR_1V', 'alias': 'TUMOR_1V', 'type': 'numeric', 'position': '7'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '65f044b261584f9e85fc391a', 'field': 'TUMOR_2V', 'alias': 'TUMOR_2V', 'type': 'numeric', 'position': '8'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '65f044b261584f9e85fc391b', 'field': 'TUMOR_3V', 'alias': 'TUMOR_3V', 'type': 'numeric', 'position': '9'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '65f044b261584f9e85fc391c', 'field': 'SURGPRIM', 'alias': 'SURGPRIM', 'type': 'numeric', 'position': '10'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '65f044b261584f9e85fc391d', 'field': 'RADIATN', 'alias': 'RADIATN', 'type': 'numeric', 'position': '11'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '65f044b261584f9e85fc391e', 'field': 'RAD_SURG', 'alias': 'RAD_SURG', 'type': 'numeric', 'position': '12'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '65f044b261584f9e85fc391f', 'field': 'o_dth_cl', 'alias': 'o_dth_cl', 'type': 'numeric', 'position': '13'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '65f044b261584f9e85fc3920', 'field': 'SURV_TM', 'alias': 'SURV_TM', 'type': 'real', 'position': '14'}], 'server': 'https://dnm.bfirst.ai:443', 'server_url': '/nexusMax/NexusMaxPlatform/uploads/platform/', 'delimiter': ',', 'file_type': 'Delimeted', 'filename': 'TumorDatasetBinned.csv', 'token': '', 'domain': '', 'is_header': 'Use Header Line', 'url': '/FileStore/platform/uploadedSourceFiles/part-00000-0eb244e5-7a6b-406d-9a07-ebd508f1aa90-c000.csv', 'results_url': 'http://dnm.bfirst.ai:44040/api/read/hdfs'}")

except Exception as ex: 
	logging.error(ex)
